In [1]:
import os, random, time
import prophet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from scipy.stats import mode
#from datetime import date
#import codecs

Importing plotly failed. Interactive plots will not work.


In [2]:
f_list = os.listdir('./data/')
f_list.sort()
print(f_list)
dt_info = np.vstack((np.column_stack((np.repeat(2020,53),np.arange(1,54))), np.column_stack((np.repeat(2021,21),np.arange(1,22)))))
end_info= 66 ## '21 WW13'

def last_calendar_week(year):
    _year, week, _day = datetime.date(year, 12, 31).isocalendar()
    return week

['4_P34_생산실적_2020.csv', '5_P34_생산계획_210405.csv', '6_P34_재고실적_210415.csv', 'DMF0035_MktFcstWaterfall(Mkt Fcst Waterfall)_20210412132805.csv', 'DMF0035_MktFcstWaterfall(Mkt Fcst Waterfall)_20210412174616.csv', 'X_Product.csv', 'X_Sales.csv', 'Y_Inven.csv']


#### 재고실적 ####

In [3]:
tmp_file = pd.read_csv('./data/'+f_list[2], encoding='euc-kr', thousands=',')
#tmp_title = ['DATE', 'MONTH', 'WW','DAY', 'IN_CELL', 'OUT_CELL', 'INVENTORY_CELL', 'IN_PACK_D', 'OUT_PACK_D', 'INVENTORY_PACK_D']                
tmp_Y_inven = pd.DataFrame({'YEAR' : tmp_file['DATE'].str.slice(0,4).astype(int), #pd.to_datetime(tmp_file['DATE']).dt.isocalendar().year,
                            'WW' : pd.to_datetime(tmp_file['DATE'], format='%Y-%m-%d').dt.isocalendar().week,
                            'DATE' : tmp_file['DATE'],
                            'INVENTORY_PACK_D' : tmp_file['INVENTORY_PACK_D']})

#tmp_inven_qty = list()
for i in range(0, end_info):
    
    if last_calendar_week(dt_info[i][0]) == dt_info[i][1]:    
        tmp_y_dta = tmp_Y_inven[(tmp_Y_inven['WW']==dt_info[i][1])] #(tmp_Y_inven['YEAR']==dt_info[i][0]) & 
    else:
        tmp_y_dta = tmp_Y_inven[(tmp_Y_inven['YEAR']==dt_info[i][0]) & (tmp_Y_inven['WW']==dt_info[i][1])] #

    #tmp_inven_qty.append(tmp_y_dta['INVENTORY_PACK_D'].mean())
    d_week = np.array(pd.to_datetime(tmp_y_dta['DATE'], format='%Y-%m-%d').dt.dayofweek)
    #tmp_y_dta['INVENTORY_PACK_D']#.iloc[0,:]#[0, (np.where(d_week == 6)[0][0]-1)]
    for j in range(0 ,len(d_week)):
        if d_week[j] == 6:
            tmp_inven_qty = tmp_y_dta['INVENTORY_PACK_D'].iloc[j]
    tmp_inven_qty = np.array((dt_info[i][0], dt_info[i][1], tmp_inven_qty))
    if i==0:
        inven_dta_qty = tmp_inven_qty
    else:
        inven_dta_qty = np.vstack((inven_dta_qty, tmp_inven_qty))
    #print(dt_info[i], tmp_y_dta.shape)
cnt = 1
for i in range(9, end_info):
    tmp_inven_qty = []
    for j in range(0, 9):
        tmp_inven_qty.append(inven_dta_qty[(i-j),2])
    if cnt ==1 :
        tmp_inven_dta = tmp_inven_qty
    else:
        tmp_inven_dta = np.vstack((tmp_inven_dta, tmp_inven_qty))
    cnt+=1
    
#inven_dta = pd.DataFrame(dt_info[9:end_info])
##inven_dta['INVENTORY_PACK_D'] = tmp_inven_qty
inven_dta = pd.DataFrame(dt_info[9:end_info])
inven_dta = pd.concat([inven_dta, pd.DataFrame(tmp_inven_dta)], axis=1)
#inven_dta 
inven_dta.columns = list(['YEAR', 'WW']) + list(map(lambda x : "INVENTORY" if x == 0  else "prev_INVENTORY_"+str(x), range(0, 9))) #list(['YEAR', 'WW', 'INVENTORY_PACK_D'])

In [4]:
inven_dta.to_csv('./data/Y_Inven.csv', index=False)

#### 생산실적 / 생산계획 ####

In [15]:
tmp_file = pd.read_csv('./data/'+f_list[1], thousands=',')
tmp_title = ['PLAN_START_YYYYMMDD','YYYYMMDD', 'QTY']
X_product_plan = tmp_file[tmp_title].copy() 
X_product_plan['PLAN_WW'] = pd.to_datetime(tmp_file['PLAN_START_YYYYMMDD'], format='%Y%m%d').dt.isocalendar().week
X_product_plan['PLAN_YEAR'] = pd.to_datetime(tmp_file['PLAN_START_YYYYMMDD'], format='%Y%m%d').dt.isocalendar().year
X_product_plan['TARGET_WW'] = pd.to_datetime(tmp_file['YYYYMMDD'], format='%Y%m%d').dt.isocalendar().week
X_product_plan['TARGET_YEAR'] = pd.to_datetime(tmp_file['YYYYMMDD'], format='%Y%m%d').dt.isocalendar().year
X_product_plan.head()

prev_name = list(map(lambda x : 'Prev_Product_'+str(x), np.arange(1, 9, 1)))
post_name = list(map(lambda x : 'Post_Product_'+str(x), np.arange(1, 9, 1)))
var_name  = list(map(lambda x : 'Var_Product_'+str(x),  np.arange(1, 9, 1)))

In [20]:
tmp_file[tmp_file['PLAN_START_YYYYMMDD']==20200308]

,DIVISION_ID,PLAN_START_YYYYMMDD,PLAN_TYPE,SEQ,OPERATION_GROUP,SCM_SITE_ID,ITEM_ID,LINE_ID,OPERATION_ID,MEASURE,YYYYMMDD,WW,YYYYMM,QTY


In [24]:
print(X_product_plan[(X_product_plan['PLAN_WW']==10) & (X_product_plan['TARGET_WW']==18) & (X_product_plan['PLAN_YEAR']==2020)])
print(X_product_plan[(X_product_plan['PLAN_WW']==10) & (X_product_plan['TARGET_WW']==18) & (X_product_plan['PLAN_YEAR']==2020)].iloc[:,2].sum())

      PLAN_START_YYYYMMDD  YYYYMMDD  QTY  PLAN_WW  PLAN_YEAR  TARGET_WW  \
2637             20200302  20200427  154       10       2020         18   
2638             20200302  20200428  154       10       2020         18   
2639             20200302  20200429  154       10       2020         18   
2640             20200302  20200430  154       10       2020         18   
2641             20200302  20200501  154       10       2020         18   
2642             20200302  20200502  154       10       2020         18   
2643             20200302  20200503  156       10       2020         18   

      TARGET_YEAR  
2637         2020  
2638         2020  
2639         2020  
2640         2020  
2641         2020  
2642         2020  
2643         2020  
1080


In [5]:
tmp_file = pd.read_csv('./data/'+f_list[0], encoding='euc-kr', thousands=',')
tmp_title = ['Prod Type','Material', 'Basic Date', 'Plan Date', 'Plan Qty', 'Result Qty']
tmp_p_real1 = tmp_file[tmp_title].copy()
tmp_p_real1['YEAR'] = pd.to_datetime(tmp_file['Plan Date']).dt.isocalendar().year
tmp_p_real1['WW'] = pd.to_datetime(tmp_file['Plan Date']).dt.isocalendar().week
X_product_actual = tmp_p_real1[(tmp_p_real1['YEAR']==2020)& (tmp_p_real1['Material']=='APVCCCMA0-A2-A01')]  #(tmp_p_real1['Prod Type']=='MASS') & 

cnt = 0
r_QTY_sum = list()
for t in range(0, end_info): #53):# 
    TW=dt_info[t,1]; TY=dt_info[t,0]    
    if any((X_product_actual['YEAR']==TY) & (X_product_actual['WW']==TW)):
        r_QTY_sum.append(X_product_actual[(X_product_actual['YEAR']==TY) & (X_product_actual['WW']==TW)]['Result Qty'].sum())
    else:
        r_QTY_sum.append(None)

        
################################################################################################
r_QTY_sum[53:66] = [842,1285,1254,1102,1274,1243,1327,870,1077,1351,1170,837,992]
################################################################################################
#product_dta['Products'] = r_QTY_sum

#### 생산계획 ####
tmp_file = pd.read_csv('./data/'+f_list[1], thousands=',')
tmp_title = ['PLAN_START_YYYYMMDD','YYYYMMDD', 'QTY']
X_product_plan = tmp_file[tmp_title].copy() 
X_product_plan['PLAN_WW'] = pd.to_datetime(tmp_file['PLAN_START_YYYYMMDD'], format='%Y%m%d').dt.isocalendar().week
X_product_plan['PLAN_YEAR'] = pd.to_datetime(tmp_file['PLAN_START_YYYYMMDD'], format='%Y%m%d').dt.isocalendar().year
X_product_plan['TARGET_WW'] = pd.to_datetime(tmp_file['YYYYMMDD'], format='%Y%m%d').dt.isocalendar().week
X_product_plan['TARGET_YEAR'] = pd.to_datetime(tmp_file['YYYYMMDD'], format='%Y%m%d').dt.isocalendar().year
X_product_plan.head()

prev_name = list(map(lambda x : 'Prev_Product_'+str(x), np.arange(1, 9, 1)))
post_name = list(map(lambda x : 'Post_Product_'+str(x), np.arange(1, 9, 1)))
var_name  = list(map(lambda x : 'Var_Product_'+str(x),  np.arange(1, 9, 1)))

cnt = 0
for t in range(9, end_info): #53):# 
    QTY_prev = list()
    QTY_post = list()    
    QTY_var = list()    
    
    for j in range(1, 9):        
        target_y = dt_info[t][0] ; target_w = dt_info[t][1]
        post_y = dt_info[t+j][0]; post_w = dt_info[t+j][1]; 
        #prev_y = dt_info[t-j][0]; prev_w = dt_info[t-j][1];        
        #print(post_y,post_w, " / ", prev_y, prev_w)
        #QTY_prev.append(X_product_plan[(X_product_plan['PLAN_YEAR']==prev_y) & (X_product_plan['PLAN_WW']==prev_w) & (X_product_plan['TARGET_YEAR']==target_y) & (X_product_plan['TARGET_WW']==target_w)]['QTY'].sum())        
        QTY_prev.append(r_QTY_sum[t-j])
        QTY_post.append(X_product_plan[(X_product_plan['PLAN_YEAR']==target_y) & (X_product_plan['PLAN_WW']==target_w) & (X_product_plan['TARGET_YEAR']==post_y) & (X_product_plan['TARGET_WW']==post_w)]['QTY'].sum())

    for j in range(1, 9):        
        target_y = dt_info[t][0] ; target_w = dt_info[t][1]
        prev_y = dt_info[t-j][0]; prev_w = dt_info[t-j][1];        
        #print(post_y,post_w, " / ", prev_y, prev_w)
        QTY_var.append(X_product_plan[(X_product_plan['PLAN_YEAR']==prev_y) & (X_product_plan['PLAN_WW']==prev_w) & (X_product_plan['TARGET_YEAR']==target_y) & (X_product_plan['TARGET_WW']==target_w)]['QTY'].sum())

    tmp_QTY = QTY_prev + QTY_post + QTY_var   
    if cnt== 0:
        tmp_X_product_plan = tmp_QTY
    else:
        tmp_X_product_plan = np.vstack((tmp_X_product_plan, tmp_QTY))
    cnt+=1
    
product_dta = pd.DataFrame(tmp_X_product_plan)
product_dta.columns = list(prev_name+post_name+var_name)
product_dta['YEAR'] = dt_info[9:end_info,0]
product_dta['WW'] = dt_info[9:end_info,1]
product_dta['Products'] = r_QTY_sum[9:end_info]

In [6]:
product_dta= product_dta.fillna(-999)
product_dta= product_dta.replace(0, -999)
product_dta.to_csv('./data/X_Product.csv', index=False)

In [7]:
### 판매계획/실적 Data Set ####
def dta_merge(t_tmp, t_title):
    #t_tmp = tmp1 ; t_title = tmp1.columns
    ver_col = np.where(t_tmp.columns == 'Version')[0][0]
    t_tmp1 = pd.DataFrame(t_tmp.iloc[:,ver_col].copy())
    t_tmp1 = pd.concat([t_tmp1,t_tmp.iloc[:, 14:]], axis=1)
    return t_tmp1


tmp1_title = pd.read_csv('./data/'+f_list[3], skiprows=0, nrows=1)#, header=None)
tmp1 = pd.read_csv('./data/'+f_list[3], skiprows=2, thousands=',')
#tmp1.columns = tmp1_title.loc[0]
#tmp1_dta = column_merge(tmp1, tmp1_title)

tmp2_title = pd.read_csv('./data/'+f_list[4], skiprows=0, nrows=1)#, header=None)
tmp2 = pd.read_csv('./data/'+f_list[4], skiprows=2, thousands=',')
#tmp1_sub.columns = tmp1_sub_title.loc[0]
#tmp1_sub_dta = column_merge(tmp1_sub, tmp1_sub_title)

tmp1.columns = tmp1_title.loc[0]
tmp2.columns = tmp2_title.loc[0]

tmp1_name = list(map(lambda i : str('Y')+tmp1_title.columns[i].split('-')[0]+'_'+tmp1_title.iloc[0,i], range(14, len(tmp1_title.loc[0]))))
tmp2_name = list(map(lambda i : str('Y')+tmp2_title.columns[i].split('-')[0]+'_'+tmp2_title.iloc[0,i], range(14, len(tmp2_title.loc[0]))))
tmp1.columns = list(tmp1.columns[:14]) + tmp1_name
tmp2.columns = list(tmp2.columns[:14]) + tmp2_name

tmp1 = dta_merge(tmp1, tmp1.columns)
tmp2 = dta_merge(tmp2, tmp2.columns)

prev_name = list(map(lambda x : 'Prev_Sale_'+str(x), np.arange(8, 0, -1)))
post_name = list(map(lambda x : 'Post_Sale_'+str(x), np.arange(1, 9,  1)))
var_name  = list(map(lambda x : 'Var_Sale_'+str(x), np.arange(1, 9,  1)))

sales_x = []
sales_y = []

for l in range(9, end_info):#dt_info.shape[0]):
    max_week = last_calendar_week(dt_info[l][0])
    check_week = str('Y')+str(dt_info[l][0])+'_'+str(max_week)
    if (dt_info[l][1] + 8 < max_week) & (dt_info[l][0] == 2020):
        tmp_dta = tmp1
        tmp_name = list(map(lambda i : str('Y')+tmp1_title.columns[i].split('-')[0]+'_'+tmp1_title.iloc[0,i], range(14, len(tmp1_title.loc[0]))))
        #print('tmp1')
    else:
        tmp_dta = tmp2
        tmp_name = list(map(lambda i : str('Y')+tmp2_title.columns[i].split('-')[0]+'_'+tmp2_title.iloc[0,i], range(14, len(tmp2_title.loc[0]))))
        if 'Y2021_W53' in tmp_name :
            re_col = tmp_name.index('Y2021_W53')#np.where('Y2021_W53' == tmp_name)[0]
            tmp_name[re_col] = 'Y2020_W53'        
        #print('tmp2')

    if check_week in tmp_name:
        check_index = tmp_name.index(check_week)
        tmp_name[check_index] = (str('Y')+str(dt_info[l][0])+'_W'+str(max_week))
    tmp_dta.columns = list(tmp_dta.columns[:1]) + tmp_name
    #print(tmp_name)
    tmp_row_y = pd.to_datetime(tmp_dta['Version'].str.slice(5,13), format='%Y%m%d').dt.isocalendar().year
    tmp_row_w = pd.to_datetime(tmp_dta['Version'].str.slice(5,13), format='%Y%m%d').dt.isocalendar().week
    #print(l, dt_info[l])
    current_no = np.where((dt_info[l][0] == tmp_row_y) & (dt_info[l][1] == tmp_row_w))[0][0]
    prev_no = l - 8
    post_no = (l+1) + 8

    prev_x = list(map(lambda i : (str('Y')+str(dt_info[i][0])+'_W0'+str(dt_info[i][1])) if np.divmod(dt_info[i][1], 10)[0] == 0 else (str('Y')+str(dt_info[i][0])+'_W'+str(dt_info[i][1])), range(prev_no, l)))
    post_x = list(map(lambda i : (str('Y')+str(dt_info[i][0])+'_W0'+str(dt_info[i][1])) if np.divmod(dt_info[i][1], 10)[0] == 0 else (str('Y')+str(dt_info[i][0])+'_W'+str(dt_info[i][1])), range((l+1), post_no)))
    current_x = (str('Y')+str(dt_info[l][0])+'_W0'+str(dt_info[l][1])) if np.divmod(dt_info[l][1], 10)[0] == 0 else (str('Y')+str(dt_info[l][0])+'_W'+str(dt_info[l][1]))
    
    tmp_sale_list = []
    tmp_col = np.where(tmp_dta.columns == current_x)[0]
    if (current_no+1) < tmp_dta.shape[0]:
        if len(tmp_col) == 1:        
            tmp_sale_list.append(mode(tmp_dta.iloc[(current_no+1):, tmp_col]).mode[0][0])
        else:        
            tmp_sale_list.append(mode(tmp_dta.iloc[(current_no+1):, tmp_col].fillna(0).sum(axis=1)).mode[0])
    else:
        tmp_sale_list.append(0)

    tmp_prev_list = []
    for i in prev_x:        
        tmp_col = np.where(tmp_dta.columns == i)[0]
        if len(tmp_col) == 1:
            tmp_prev_list.append(tmp_dta.iloc[current_no, tmp_col].values[0])        
        else:
            tmp_prev_list.append(tmp_dta.iloc[current_no, tmp_col].fillna(0).sum())

    tmp_post_list = []
    for i in post_x:        
        tmp_col = np.where(tmp_dta.columns == i)[0]
        if len(tmp_col) == 1:
            tmp_post_list.append(tmp_dta.iloc[current_no, tmp_col].values[0])        
        else:
            tmp_post_list.append(tmp_dta.iloc[current_no, tmp_col].fillna(0).sum())

    tmp_var_list = []
    for i in range(0, 8):    
        tmp_col = np.where(tmp_dta.columns == current_x)[0]        
        if len(tmp_col) == 1:
            tmp_var_list.append(tmp_dta.iloc[((current_no-1) - i), tmp_col].values[0])
        else:
            tmp_var_list.append(tmp_dta.iloc[((current_no-1) - i), tmp_col].fillna(0).sum())
    #print(tmp_var_list)
    tmp_x1 = list(tmp_prev_list + tmp_post_list + tmp_var_list)
    tmp_x2 = tmp_sale_list[0]
    sales_x.append(tmp_x1)
    sales_y.append(tmp_x2)
sales_dta = pd.DataFrame(sales_x)

sales_dta.columns = list(prev_name+post_name+var_name)#+['Sale'])
sales_dta['YEAR'] = dt_info[9:end_info, 0]
sales_dta['WW'] = dt_info[9:end_info,1]
sales_dta['Sales'] = sales_y

In [8]:
sales_dta = sales_dta.fillna(-999)
sales_dta = sales_dta.replace(0, -999)
sales_dta = sales_dta.replace(-2, -999)
sales_dta = sales_dta.replace(-4, -999)
sales_dta.to_csv('./data/X_Sales.csv', index=False)

In [13]:
#pd.to_datetime(tmp1['Version'].str.slice(5,13), format='%Y%m%d').dt.isocalendar().week
tmp1['Version'].loc[8]

'MF_W_20200303_V001'